# Парсер новостей с HABR.COM на PYTHON

In [1]:
# Подключаем библиотеку для поиска элементов кода на сайте !!!!!!!!!!!!!
# Библиотека PYTHON для извлечения данных из файлов HTML и XML
from bs4 import BeautifulSoup

# Подключаем библиотеку для парсинга сайтов !!!!!!!!!!!!!
import requests

In [1]:
# Подключаем библиотеку для поиска элементов кода на сайте !!!!!!!!!!!!!
# Библиотека PYTHON для извлечения данных из файлов HTML и XML
from bs4 import BeautifulSoup

# Подключаем библиотеки для парсинга сайтов !!!!!!!!!!!!!
import requests
import lxml
import os
import json
import csv

# Работа с файлами XLSX
import openpyxl
import xlsxwriter

# Использование объекта datetime :
from datetime import datetime
from time import sleep
import time
import random

In [2]:
# Переменная со ссылкой на сайт
url = 'https://vc.ru/popular'

In [ ]:
# Переменная со ссылкой на сайт
url = 'https://vc.ru/new'

# ФУНКЦИЯ № 1. Определение переменных

In [2]:
def variable():
# 0. Создаём словарь заголовков
    headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
        }
    
# 1. Список адресов основного хоста
    hosts = ['https://habr.com']
    
# 2. Список адресов страниц для парсинга
    urls = ['https://habr.com/ru/news/']
# url_2 = 'https://habr.com/ru/news/page2/'

# 3. Список имен сайтов - список словарей
    names = [{"name": u.split('//')[1].split('.')[0]} for u in urls]
    
# 4. Словарь путей для папок с данными
    type_files = ['csv', 'html', 'jpg', 'json', 'txt', 'xlsx']
    path_files = ['../data/processed/csv/', '../data/interim/html/', '../data/processed/img/',
                 '../data/processed/json/', '../data/processed/txt/', '../data/processed/xlsx/'
                ]
    extension_files = ['.csv', '.html', '.jpg', '.json', '.txt', '.xlsx']
    
    paths = [{'type_file': type_files[i], 'path_file': path_files[i], 'extension_file': extension_files[i]} for i in range(len(type_files))]
    
# 5. Словарь атрибутов Супа - список списков
    divs = ['div']
    classes = ['tm-article-snippet']
    pattern = [[divs[i], classes[i]] for i in range(len(divs))]
    
    
    return headers, hosts, urls, names, paths, pattern

In [7]:
print(variable()[3][0]['name'])
# print(variable()[3][0])

vc


# ФУНКЦИЯ № 2. Сохранение HTML-кода для тестирования

In [3]:
# import os
# Отправляем запрос на страницу, передаём в качестве GET-параметра URL-адрес
# Сохраним нашу страницу в файл И ЗАКОММЕНТИРУЕМ ЭТУ КОМАНДУ за ненадобностью

def write_html(headers, url, path):
    # path = '../data/interim/html/'
# Если папки для сохранения результатов не существует, то она будет создана
    if not os.path.isdir(path[1]['path_file']):
        print(f'Fun.2.0. Folder "{path[1]['path_file']}" does NOT exist - it will be created !!!')
        os.mkdir(path[1]['path_file'])
    else:
        print(f'Fun.2.0. Folder "{path[1]['path_file']}" FOUND!!! It`s OK')
    
    print(f'Fun.2.0. START of function "WRITE_HTML()"\n')
    
    count = 1
# Проходим циклом по списку со ссылками на страницы сайтов
    for u in url:
        r = requests.get(url=u, headers=headers)
        src = r.text

        file_start = 'news_articles_'
        name = u.split('//')[1].split('.')[0]
        file_name = file_start + name
        file_path = os.path.join(path[1]['path_file'], file_name) + path[1]['extension_file']
        print(f'{count}. Function "WRITE_HTML({name})" EXECUTING ...')

# Проверка существования старых файлов с таким же названием (если они существуют - обновляем)
        if not os.path.exists(file_path):
            print(f"\tThere is NO file \"{file_name}\" in the folder \"{path[1]['path_file']}\" - It will be CREATED.")
        else:
            print(f"\tIn the folder \"{path[1]['path_file']}\" the file \"{file_name}\" EXISTS.")
            print(f'\tThe file "{file_name}" will be DELETED and REWRITTEN.')
            os.remove(file_path)
            
# Сохраняем код по указанному пути
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(src)
        print(f'\tHTML-code \"{file_name}\" is SAVED in folder \"{path[1]['path_file']}\"')
        print(f'Function "WRITE_HTML({name.upper()})" is EXECUTED\n')
        count += 1

    print(f'Fun.2.0. Function "WRITE_HTML()" COMPLETED successfully!')

# Безусловный запуск функции
if __name__ == '__main__':
    write_html(variable()[0], variable()[2], variable()[4])

Fun.2.0. Folder "../data/interim/html/" FOUND!!! It`s OK
Fun.2.0. START of function "WRITE_HTML()"

1. Function "WRITE_HTML(habr)" EXECUTING ...
	There is NO file "news_articles_habr" in the folder "../data/interim/html/" - It will be CREATED.
	HTML-code "news_articles_habr" is SAVED in folder "../data/interim/html/"
Function "WRITE_HTML(HABR)" is EXECUTED

Fun.2.0. Function "WRITE_HTML()" COMPLETED successfully!


# ФУНКЦИЯ № 3.0. Чтение HTML-кода, сохранение его в список

In [4]:
# READ_HTML:
def read_html(name, path):
    print(f'Fun.3.0. START of function "READ_HTML()"')
    count = 1
    
    # print(name)
    file_start = 'news_articles_'
    file_name = file_start + name
    file_path = os.path.join(path['path_file'], file_name) + path['extension_file']
    print(f'{count}. Function "READ_HTML({name.upper()})" is EXECUTING...')
    
# Проверка существования файлов с HTML-кодом. Существующие коды собираем в список.
    if os.path.exists(file_path):
        print(f"\tIn the folder \"{path['path_file']}\" the file \"{file_name}\" EXISTS.")
        
        with open(file_path, encoding='utf-8') as file:
            src = file.read()
            
        print(f'\tHTML-code "{file_name}" is OPENED from {file_path}')
    else:
        print(f"\tThere is NO file \"{file_name}\" in the \"{path['path_file']}\" folder!")
        print('\tIt is necessary to START function "WRITE_HTML()" !!!\n')
        # break
        
    count += 1
    print(f'Fun.3.0. "READ_HTML({name.upper()})" COMPLETED SUCCESSFULLY!\n')

    return src

# Безусловный запуск функции
if __name__ == '__main__':
    read_html(variable()[3][0]['name'], variable()[4][1])

Fun.3.0. START of function "READ_HTML()"
1. Function "READ_HTML(HABR)" is EXECUTING...
	In the folder "../data/interim/html/" the file "news_articles_habr" EXISTS.
	HTML-code "news_articles_habr" is OPENED from ../data/interim/html/news_articles_habr.html
Fun.3.0. "READ_HTML(HABR)" COMPLETED SUCCESSFULLY!



# ФУНКЦИЯ № 7.0. Создание CSV-файла

In [28]:
# Для заполнения CSV файла содержимым построчно - def save_csv(name, count, title, link, date_time, path):
def save_csv(name, count, author, date, title, shorts, link, date_time, path):
    
    file_start = 'news_articles_'
    file_name = file_start + name
    file_path = os.path.join(path['path_file'], file_name) + path['extension_file']
    
# Заполняем CSV файл построчно (внутри цикла); "А" - APPEND - добавление записей 
    with open(file_path, 'a', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(
            (
                count,
                author,
                date,
                title,
                shorts,
                link,
                date_time
            )
        )

In [3]:
# Создаём запрос
# response = requests.get(url)

In [5]:
# Объект SOUP
# soup = BeautifulSoup(response.text, 'html')
soup = BeautifulSoup(read_html(variable()[3][0]['name'], variable()[4][1]), 'lxml')

# Вылезает куча кода в виде текста
# print(soup)

Fun.3.0. START of function "READ_HTML()"
1. Function "READ_HTML(HABR)" is EXECUTING...
	In the folder "../data/interim/html/" the file "news_articles_habr" EXISTS.
	HTML-code "news_articles_habr" is OPENED from ../data/interim/html/news_articles_habr.html
Fun.3.0. "READ_HTML(HABR)" COMPLETED SUCCESSFULLY!



In [9]:
# Определяем DIV-ы с нужной инфой
# articles = soup.find('div', class_ = 'content content--short')
div = soup.find('div', class_ = 'tm-article-snippet')
print(div)

<div class="tm-article-snippet tm-article-snippet"><!--[--><!--]--><div class="tm-article-snippet__meta-container"><div class="tm-article-snippet__meta"><span class="tm-user-info tm-article-snippet__author"><a class="tm-user-info__userpic" data-test-id="user-info-pic" href="/ru/users/mefdayy/" title="mefdayy"><div class="tm-entity-image"><!--[--><img alt="" class="tm-entity-image__pic" height="24" src="https://assets.habr.com/habr-web/img/avatars/152.png" width="24"/><!--]--></div></a><span class="tm-user-info__user tm-user-info__user_appearance-default" data-test-id="user-info-description"><a class="tm-user-info__username" href="/ru/users/mefdayy/">mefdayy <!-- --></a><!--[--><a class="tm-article-datetime-published tm-article-datetime-published_link" href="/ru/companies/bothub/news/884024/"><!--[--><time datetime="2025-02-19T15:39:28.000Z" title="2025-02-19, 18:39">только что</time><!--]--></a><!--]--></span></span></div><!-- --></div><h2 class="tm-title tm-title_h2" data-test-id="art

In [21]:
# Определяем DIV-ы с нужной инфой - ВАРИАНТ С ПАТТЕРНОМ
pattern = variable()[5][0]
div = soup.find(pattern[0], class_= pattern[1])
print(div)

<div class="tm-article-snippet tm-article-snippet"><!--[--><!--]--><div class="tm-article-snippet__meta-container"><div class="tm-article-snippet__meta"><span class="tm-user-info tm-article-snippet__author"><a class="tm-user-info__userpic" data-test-id="user-info-pic" href="/ru/users/mefdayy/" title="mefdayy"><div class="tm-entity-image"><!--[--><img alt="" class="tm-entity-image__pic" height="24" src="https://assets.habr.com/habr-web/img/avatars/152.png" width="24"/><!--]--></div></a><span class="tm-user-info__user tm-user-info__user_appearance-default" data-test-id="user-info-description"><a class="tm-user-info__username" href="/ru/users/mefdayy/">mefdayy <!-- --></a><!--[--><a class="tm-article-datetime-published tm-article-datetime-published_link" href="/ru/companies/bothub/news/884024/"><!--[--><time datetime="2025-02-19T15:39:28.000Z" title="2025-02-19, 18:39">только что</time><!--]--></a><!--]--></span></span></div><!-- --></div><h2 class="tm-title tm-title_h2" data-test-id="art

In [10]:
# Чтобы определить степени вложенности DIV-ов применяем трюк PRETTIFY:
print(div.prettify())

<div class="tm-article-snippet tm-article-snippet">
 <!--[-->
 <!--]-->
 <div class="tm-article-snippet__meta-container">
  <div class="tm-article-snippet__meta">
   <span class="tm-user-info tm-article-snippet__author">
    <a class="tm-user-info__userpic" data-test-id="user-info-pic" href="/ru/users/mefdayy/" title="mefdayy">
     <div class="tm-entity-image">
      <!--[-->
      <img alt="" class="tm-entity-image__pic" height="24" src="https://assets.habr.com/habr-web/img/avatars/152.png" width="24"/>
      <!--]-->
     </div>
    </a>
    <span class="tm-user-info__user tm-user-info__user_appearance-default" data-test-id="user-info-description">
     <a class="tm-user-info__username" href="/ru/users/mefdayy/">
      mefdayy
      <!-- -->
     </a>
     <!--[-->
     <a class="tm-article-datetime-published tm-article-datetime-published_link" href="/ru/companies/bothub/news/884024/">
      <!--[-->
      <time datetime="2025-02-19T15:39:28.000Z" title="2025-02-19, 18:39">
       т

In [13]:
# author__name
author = div.find('a', class_='tm-user-info__username').text.strip()
print(author)

mefdayy 


In [12]:
# Достаём текст заголовка из этого DIV-а
# title = div.text
title = div.find('h2', class_='tm-title_h2').text.strip()
print(title)

Логические модели OpenAI получили два полезных обновления


In [15]:
# Краткое содержание
shorts = div.find('div', class_='article-formatted-body').text.strip()
print(shorts)
# block-text

Модели рассуждений OpenAI, o1 и o3-mini, являются полезными инструментами для тех, кто использует ChatGPT для более сложных задач, включая программирование, математику и даже многоэтапные текстовые задачи, такие как решение проблемы или составление маршрута. Чтобы сделать работу с программой более полезной, OpenAI внедрила небольшие, но важные обновления.


In [18]:
# Ссылку ищем как элемент словаря:
host = variable()[1][0]
link = host + div.find('h2', class_='tm-title_h2').find('a')['href']
# link = articles.a['href']

print(link)
# print(title, link)

https://habr.com/ru/companies/bothub/news/884024/


In [19]:
# Время публикации:
time_pub = div.find('a', class_ = 'tm-article-datetime-published')
print(time_pub)

<a class="tm-article-datetime-published tm-article-datetime-published_link" href="/ru/companies/bothub/news/884024/"><!--[--><time datetime="2025-02-19T15:39:28.000Z" title="2025-02-19, 18:39">только что</time><!--]--></a>


In [20]:
# Время публикации:
# time_pub = div.find('a', class_ = 'noshrink').['time datetime']
# time_pub = div.find('div', class_ = 'content-header-details__inner').find('time')
time_dt = div.find('a', class_ = 'tm-article-datetime-published').find('time')['datetime']
time_t = div.find('a', class_ = 'tm-article-datetime-published').find('time')['title']
print(time_dt, '-', time_t)
# print(title, link)

2025-02-19T15:39:28.000Z - 2025-02-19, 18:39


## ЭТО СКАЧАННЫЙ КОД !!!

In [22]:
# Достаём все заголовки статей со ссылками (используем цикл):
# Объект SOUP
soup = BeautifulSoup(read_html(variable()[3][0]['name'], variable()[4][1]), 'lxml')
HOST = variable()[1][0]
PATTERN = variable()[5][0]

# divs = soup.find_all('div', class_ = 'tm-article-snippet')
divs = soup.find_all(PATTERN[0], class_= PATTERN[1])
count = 1
for art in divs:
    author = art.find('a', class_='tm-user-info__username').text.strip()
    date = art.find('a', class_ = 'tm-article-datetime-published').find('time')['title']
    title = art.find('h2', class_='tm-title_h2').text.strip() if art.find('h2', class_='tm-title_h2').text.strip() else 'Absent'
    shorts = art.find('div', class_='article-formatted-body').text.strip() if art.find('div', class_='article-formatted-body').text.strip() else 'Absent'
    link = HOST + div.find('h2', class_='tm-title_h2').find('a')['href']
    date_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    print(f'{count}. {author} - {date}\n{title}\n{shorts}\n{link}\n{date_time}\n')
    count +=1

Fun.3.0. START of function "READ_HTML()"
1. Function "READ_HTML(HABR)" is EXECUTING...
	In the folder "../data/interim/html/" the file "news_articles_habr" EXISTS.
	HTML-code "news_articles_habr" is OPENED from ../data/interim/html/news_articles_habr.html
Fun.3.0. "READ_HTML(HABR)" COMPLETED SUCCESSFULLY!

1. mefdayy - 2025-02-19, 18:39
Логические модели OpenAI получили два полезных обновления
Модели рассуждений OpenAI, o1 и o3-mini, являются полезными инструментами для тех, кто использует ChatGPT для более сложных задач, включая программирование, математику и даже многоэтапные текстовые задачи, такие как решение проблемы или составление маршрута. Чтобы сделать работу с программой более полезной, OpenAI внедрила небольшие, но важные обновления.
https://habr.com/ru/companies/bothub/news/884024/
2025-02-19 19:28:08

2. mefdayy - 2025-02-19, 18:25
Alpha3D и MoonlanderAI объединяют усилия: революция в создании игр и 3D-приложений
Alpha3D, платформа для создания 3D-ресурсов на основе AI, пр

## ЭТО ИНТЕРНЕТ - ONLINE !!!

In [27]:
URL = variable()[2][0]
page=3

url_temp = URL + f'page{page}/'
print(url_temp)

https://habr.com/ru/news/page3/


In [25]:
# Рабочий вариант без скроллинга

HEADERS = variable()[0]
HOST = variable()[1][0]
URL = variable()[2][0]
PATTERN = variable()[5][0]

page = 1
count = 1

r = requests.get(url=URL, headers=HEADERS)
src = r.text
soup = BeautifulSoup(src, 'lxml')
divs = soup.find_all(PATTERN[0], class_= PATTERN[1])

print(f'PAGE={page}')
for art in divs:
    author = art.find('a', class_='tm-user-info__username').text.strip()
    date = art.find('a', class_ = 'tm-article-datetime-published').find('time')['title']
    title = art.find('h2', class_='tm-title_h2').text.strip() if art.find('h2', class_='tm-title_h2').text.strip() else 'Absent'
    shorts = art.find('div', class_='article-formatted-body').text.strip() if art.find('div', class_='article-formatted-body').text.strip() else 'Absent'
    link = HOST + div.find('h2', class_='tm-title_h2').find('a')['href']
    date_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    sleep(random.randrange(1, 3)) # Для симуляции работы реального пользователя, а не робота
    print(f'{count}. {author} - {date}\n{title}\n{shorts}\n{link}\n{date_time}\n')
    count +=1

PAGE=1
1. SLY_G - 2025-02-19, 19:35
Китай продемонстрировал первый в мире двусторонний интерфейс мозг-компьютер
Китайские учёные разработали первый в мире двусторонний адаптивный интерфейс мозг-компьютер (ИМК), значительно повышающий эффективность технологии и открывающий путь к практическому применению. Согласно новому исследованию, система работает в 100 раз эффективность по сравнению с традиционными ИМК.Исследователи из Тяньцзиньского университета и Университета Цинхуа утверждают, что их инновацию вскоре можно будет интегрировать в носимые и портативные устройства для медицинского и потребительского использования.Эта система позволяет мозгу и машине учиться друг у друга, в отличие от традиционных ИМК, которые только декодируют сигналы мозга. Такое взаимодействие обеспечивает стабильную работу с течением времени, что является ключевым шагом на пути к повышению надёжности ИМК в повседневном применении.
https://habr.com/ru/companies/bothub/news/884024/
2025-02-19 19:40:56

2. daniilsha

### ПАГИНАЦИЯ для 5 страниц 

In [29]:
# ПАГИНАЦИЯ для 5 страниц
name = 'habr'
HEADERS = variable()[0]
HOST = variable()[1][0]
URL = variable()[2][0]
path_csv = variable()[4][0]
PATTERN = variable()[5][0]

page = 1
max_page = 5
count_link = 1
articles_list = []

while page <= max_page:
    print(f'PAGE={page}')
    url_temp = URL + f'page{page}/'
    r = requests.get(url=url_temp, headers=HEADERS)
    src = r.text
    soup = BeautifulSoup(src, 'lxml')
    divs = soup.find_all(PATTERN[0], class_= PATTERN[1])
    time.sleep(2)

    for art in divs:
        author = art.find('a', class_='tm-user-info__username').text.strip()
        date = art.find('a', class_ = 'tm-article-datetime-published').find('time')['title']
        title = art.find('h2', class_='tm-title_h2').text.strip() if art.find('h2', class_='tm-title_h2').text.strip() else 'Absent'
        shorts = art.find('div', class_='article-formatted-body').text.strip() if art.find('div', class_='article-formatted-body').text.strip() else 'Absent'
        link = HOST + div.find('h2', class_='tm-title_h2').find('a')['href']
        date_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        sleep(random.randrange(1, 3)) # Для симуляции работы реального пользователя, а не робота
        print(f'{count_link}. {author} - {date}\n{title}\n{shorts}\n{link}\n{date_time}\n')
        

        # articles_list.append(
        #         {
        #             "Number": count_link,
        #             "Author": author,
        #             "Date": date,
        #             "Topic": title,
        #             "Shorts": shorts,
        #             "Link": link,
        #             "DateTime": date_time
        #         }
        #     )
        save_csv(name, count_link, author, date, title, shorts, link, date_time, path_csv)
        count_link +=1
    page += 1

PAGE=1
1. SLY_G - 2025-02-19, 22:15
Квантовые поиски источника времени не обнаружили разницы между прошлым и будущим
В надежде обнаружить источник реки времени физики из Университета Суррея Томас Гуфф, Чинталпати Умашанкар Шастри и Андреа Рокко вели поиски в эквиваленте горячей квантовой ванны.Разумеется, они не нашли того, что искали, и подтвердили, что время течёт назад так же легко, как и вперёд, в гудящем квантовом вакууме. Тем не менее, выученные нами уроки могут однажды раскрыть, почему физика настаивает на существовании истории.Физика, по большому счёту, симметрична, когда речь идёт о времени. Возможно, мы никогда не увидим, как срастается разбитое яйцо или дуб превращается в жёлудь, но если свести большинство процессов к их фундаментальному уровню, то мало что будет диктовать, какая часть уравнения должна указывать в прошлое, а какая — в будущее.
https://habr.com/ru/companies/bothub/news/884024/
2025-02-19 22:19:49

2. AnnieBronson - 2025-02-19, 22:12
«Лаборатория Касперского»:

In [31]:
count = 1
page=1

while page < 3:
    print(page)
    for i in range(1,13):
        print(count, end=', ')
        count +=1
    print()
    page+=1

1
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 
2
13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 


# FORUM

In [ ]:
# Если нужно прокрутить к целевому элементу, можно воспользоваться методом .move_to_element() из ActionChains, пример:

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

driver = webdriver.Chrome()
driver.get('http://www.w3schools.com/')
target = driver.find_element_by_link_text('BROWSE TEMPLATES')
actions = ActionChains(driver)
actions.move_to_element(target)
actions.perform()

In [ ]:
# Для прокрутки в конец (начало) страницы через .send_keys():

from selenium import webdriver
from selenium.webdriver.common.keys import Keys 

driver = webdriver.Chrome()
driver.get('http://www.w3schools.com/')
driver.find_element_by_tag_name('body').send_keys(Keys.END)

# send_keys(Keys.HOME), если в начало
# так тоже сработает:
# send_keys(Keys.DOWN) send_keys(Keys.PAGE_DOWN)
# send_keys(Keys.UP) send_keys(Keys.PAGE_UP)

In [ ]:
# Для прокрутки на заданное количество пикселей метод окна .scrollBy():

scroll_vаlue = -200
scroll_by = f'window.scrollBy(0, {scroll_vаlue});'
driver.execute_script(scroll_by)

In [ ]:
# Второй вариант цикл скроллинга

from selenium.webdriver.common.keys import Keys

while True:
    # прокручиваем страницу до конца
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    # проверяем, появились ли новые товары
    if driver.find_elements(By.CLASS_NAME, "tovar"):
        with open("C:/Users/SERGEY/PycharmProjects/ТEST/Test1", "w", encoding="utf=8") as file:
            file.write(driver.page_source)
    else:
        break